In [ ]:
!pip install -q torch transformers openai groq

import os
import torch
import math
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoTokenizer, AutoModelForSequenceClassification
from openai import OpenAI
from google.colab import userdata
from transformers import logging

logging.set_verbosity_error()

try:
    os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
    os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
    HF_TOKEN = os.getenv("HF_TOKEN")
    GROQ_API_KEY = os.getenv("GROQ_API_KEY")
except Exception as e:
    print(f"Could not retrieve secrets. Please ensure they are set in your Colab environment. Error: {e}")
    HF_TOKEN = None
    GROQ_API_KEY = None


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


def run_hf_detector(text, model_id="roberta-base-openai-detector"):
    
    tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)
    model = AutoModelForSequenceClassification.from_pretrained(model_id, token=HF_TOKEN).to(device)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
       
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1).cpu().numpy()[0]

    human_score = float(probs[0])
    ai_score = float(probs[1])
    label = "AI-generated" if ai_score > human_score else "Human-generated"
    return {"ai_score": ai_score, "human_score": human_score, "hf_label": label}

def calculate_perplexity(text):

    model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    encodings = tokenizer(text, return_tensors="pt").to(device)

    max_length = model.config.n_positions
    if encodings.input_ids.size(1) > max_length:
        encodings.input_ids = encodings.input_ids[:, :max_length]
        encodings.attention_mask = encodings.attention_mask[:, :max_length]

    with torch.no_grad():
        outputs = model(**encodings, labels=encodings.input_ids)
    loss = outputs.loss
    perplexity = math.exp(loss.item())
    label = "AI-generated" if perplexity < 60 else "Human-generated"
    return {"perplexity": perplexity, "perplexity_label": label}

def generate_explanation(text, ai_score, human_score):

    if not GROQ_API_KEY:
        return "GROQ_API_KEY not found. Cannot generate explanation."

    try:
        groq_client = OpenAI(api_key=GROQ_API_KEY, base_url="https://api.groq.com/openai/v1")
        decision = "AI-generated" if ai_score > human_score else "Human-generated"

        prompt = (
            f"You are an AI text analysis expert. Based on the following text, provide a concise, one or two-sentence explanation for why it was classified as '{decision}'. "
            "Focus on the text's style, word choice, and structure. Do not mention the scores or probabilities in your explanation.\n\n"
            f"Text: \"{text}\"\n\n"
            "Explanation:"
        )

        resp = groq_client.chat.completions.create(
            model="gemma2-9b-it",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=150,
            temperature=0.7
        )
        return resp.choices[0].message.content.strip()
    except Exception as e:
        return f"Could not generate explanation: {str(e)}"

def analyze_text(text):

    results = {}

    try:
        hf_out = run_hf_detector(text)
        results.update(hf_out)
        diff = abs(results["ai_score"] - results["human_score"])
        if diff > 0.8:
            results["confidence"] = "High"
        elif 0.3 <= diff <= 0.8:
            results["confidence"] = "Medium"
        else:
            results["confidence"] = "Low"
    except Exception as e:
        print(f"HF Detector error: {e}")
        results["hf_label"] = "Error"
        results["confidence"] = "Low"

    try:
        results.update(calculate_perplexity(text))
    except Exception as e:
        print(f"Perplexity error: {e}")
        results["perplexity_label"] = "Error"

    if "Error" not in [results.get("hf_label"), results.get("perplexity_label")]:
        if results.get("hf_label") != results.get("perplexity_label"):
            print("Notice: Detector and Perplexity models disagree. Confidence set to Low.")
            results["confidence"] = "Low"

    if "ai_score" in results:
        explanation = generate_explanation(text, results["ai_score"], results["human_score"])
        results["explanation"] = explanation
    else:
        results["explanation"] = "Could not run primary detector to generate an explanation."

    return results

sample_text = "That is an excellent and crucial point. You are absolutely right. My previous solution had a logical flaw: it detected AI-written text that was spoken aloud, not an AI-generated voice. To do what you require, we must analyze the audio itself for the electronic fingerprints of a synthetic voice, rather than analyzing the words being spoken. I have completely rewritten the audio portion of the script to do this correctly."

final_output = analyze_text(sample_text)

print("\n✅ Final Analysis:\n")

probability_score = final_output.get("ai_score", 0.0) 
confidence_level = final_output.get("confidence", "N/A")
explanation = final_output.get("explanation", "No explanation available.")

print("---")
print(f"### Probability Score")
print(f"The AI generation probability score is **{probability_score:.4f}**.")
print("\n---")
print(f"### Confidence")
print(f"The confidence level for this prediction is **{confidence_level}**.")
print("\n---")
print(f"### Explanation")
print(f"{explanation}")

Using device: cpu

✅ Final Analysis:

---
### Probability Score
The AI generation probability score is **0.5973**.

---
### Confidence
The confidence level for this prediction is **Low**.

---
### Explanation
The text exhibits a stilted and overly formal tone, with repetitive phrases like "excellent and crucial point" and "absolutely right."  The language lacks natural variation and feels overly rehearsed, suggesting an AI's attempt to mimic human conversation.
